# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jan 17 2023 10:43AM,255200,10,0086304108,ISDIN Corporation,Released
1,Jan 17 2023 10:43AM,255200,10,0086304383,ISDIN Corporation,Released
2,Jan 17 2023 10:43AM,255200,10,0086304381,ISDIN Corporation,Released
3,Jan 17 2023 10:43AM,255200,10,0086304385,ISDIN Corporation,Released
4,Jan 17 2023 10:43AM,255200,10,0086304609,ISDIN Corporation,Released
5,Jan 17 2023 10:43AM,255200,10,0086304608,ISDIN Corporation,Released
6,Jan 17 2023 10:43AM,255200,10,0086304628,ISDIN Corporation,Released
7,Jan 17 2023 10:43AM,255200,10,0086304629,ISDIN Corporation,Released
8,Jan 17 2023 10:43AM,255200,10,0086304632,ISDIN Corporation,Released
9,Jan 17 2023 10:43AM,255200,10,0086304633,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
18,255197,Released,1
19,255198,Released,1
20,255199,Released,1
21,255200,Released,15
22,255201,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
255197,NaN,NaN,1.0
255198,NaN,NaN,1.0
255199,NaN,NaN,1.0
255200,NaN,NaN,15.0
255201,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
255090,29.0,17.0,2.0
255097,0.0,41.0,3.0
255157,0.0,0.0,100.0
255172,0.0,0.0,21.0
255175,0.0,6.0,4.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
255090,29,17,2
255097,0,41,3
255157,0,0,100
255172,0,0,21
255175,0,6,4


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,255090,29,17,2
1,255097,0,41,3
2,255157,0,0,100
3,255172,0,0,21
4,255175,0,6,4


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,255090,29,17,2
1,255097,,41,3
2,255157,,,100
3,255172,,,21
4,255175,,6,4


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jan 17 2023 10:43AM,255200,10,ISDIN Corporation
15,Jan 17 2023 10:43AM,255201,10,ISDIN Corporation
16,Jan 17 2023 10:40AM,255199,12,Hush Hush
17,Jan 17 2023 10:36AM,255198,20,"HVL, LLC dba Atrium Innovations"
18,Jan 17 2023 10:36AM,255191,10,Eywa Pharma Inc.
29,Jan 17 2023 10:34AM,255190,10,Bio-PRF
49,Jan 17 2023 10:32AM,255196,10,MedStone Pharma LLC
52,Jan 17 2023 10:30AM,255197,12,Hush Hush
53,Jan 17 2023 10:28AM,255195,12,Hush Hush
54,Jan 17 2023 10:26AM,255192,10,"Nextsource Biotechnology, LLC"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jan 17 2023 10:43AM,255200,10,ISDIN Corporation,,,15
1,Jan 17 2023 10:43AM,255201,10,ISDIN Corporation,,,1
2,Jan 17 2023 10:40AM,255199,12,Hush Hush,,,1
3,Jan 17 2023 10:36AM,255198,20,"HVL, LLC dba Atrium Innovations",,,1
4,Jan 17 2023 10:36AM,255191,10,Eywa Pharma Inc.,,,11
5,Jan 17 2023 10:34AM,255190,10,Bio-PRF,,,20
6,Jan 17 2023 10:32AM,255196,10,MedStone Pharma LLC,,,3
7,Jan 17 2023 10:30AM,255197,12,Hush Hush,,,1
8,Jan 17 2023 10:28AM,255195,12,Hush Hush,,,1
9,Jan 17 2023 10:26AM,255192,10,"Nextsource Biotechnology, LLC",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 17 2023 10:43AM,255200,10,ISDIN Corporation,15,,
1,Jan 17 2023 10:43AM,255201,10,ISDIN Corporation,1,,
2,Jan 17 2023 10:40AM,255199,12,Hush Hush,1,,
3,Jan 17 2023 10:36AM,255198,20,"HVL, LLC dba Atrium Innovations",1,,
4,Jan 17 2023 10:36AM,255191,10,Eywa Pharma Inc.,11,,
5,Jan 17 2023 10:34AM,255190,10,Bio-PRF,20,,
6,Jan 17 2023 10:32AM,255196,10,MedStone Pharma LLC,3,,
7,Jan 17 2023 10:30AM,255197,12,Hush Hush,1,,
8,Jan 17 2023 10:28AM,255195,12,Hush Hush,1,,
9,Jan 17 2023 10:26AM,255192,10,"Nextsource Biotechnology, LLC",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 17 2023 10:43AM,255200,10,ISDIN Corporation,15,,
1,Jan 17 2023 10:43AM,255201,10,ISDIN Corporation,1,,
2,Jan 17 2023 10:40AM,255199,12,Hush Hush,1,,
3,Jan 17 2023 10:36AM,255198,20,"HVL, LLC dba Atrium Innovations",1,,
4,Jan 17 2023 10:36AM,255191,10,Eywa Pharma Inc.,11,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 17 2023 10:43AM,255200,10,ISDIN Corporation,15.0,NaN,NaN
1,Jan 17 2023 10:43AM,255201,10,ISDIN Corporation,1.0,NaN,NaN
2,Jan 17 2023 10:40AM,255199,12,Hush Hush,1.0,NaN,NaN
3,Jan 17 2023 10:36AM,255198,20,"HVL, LLC dba Atrium Innovations",1.0,NaN,NaN
4,Jan 17 2023 10:36AM,255191,10,Eywa Pharma Inc.,11.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 17 2023 10:43AM,255200,10,ISDIN Corporation,15.0,0.0,0.0
1,Jan 17 2023 10:43AM,255201,10,ISDIN Corporation,1.0,0.0,0.0
2,Jan 17 2023 10:40AM,255199,12,Hush Hush,1.0,0.0,0.0
3,Jan 17 2023 10:36AM,255198,20,"HVL, LLC dba Atrium Innovations",1.0,0.0,0.0
4,Jan 17 2023 10:36AM,255191,10,Eywa Pharma Inc.,11.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2296619,124.0,47.0,0.0
12,1275978,4.0,1.0,0.0
15,255157,100.0,0.0,0.0
19,510262,23.0,17.0,29.0
20,255198,1.0,0.0,0.0
22,255187,1.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2296619,124.0,47.0,0.0
1,12,1275978,4.0,1.0,0.0
2,15,255157,100.0,0.0,0.0
3,19,510262,23.0,17.0,29.0
4,20,255198,1.0,0.0,0.0
5,22,255187,1.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,124.0,47.0,0.0
1,12,4.0,1.0,0.0
2,15,100.0,0.0,0.0
3,19,23.0,17.0,29.0
4,20,1.0,0.0,0.0
5,22,1.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,124.0
1,12,Released,4.0
2,15,Released,100.0
3,19,Released,23.0
4,20,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,19,20,22
Status,,,,,,
Completed,0.0,0.0,0.0,29.0,0.0,0.0
Executing,47.0,1.0,0.0,17.0,0.0,0.0
Released,124.0,4.0,100.0,23.0,1.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,19,20,22
0,Completed,0.0,0.0,0.0,29.0,0.0,0.0
1,Executing,47.0,1.0,0.0,17.0,0.0,0.0
2,Released,124.0,4.0,100.0,23.0,1.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,19,20,22
0,Completed,0.0,0.0,0.0,29.0,0.0,0.0
1,Executing,47.0,1.0,0.0,17.0,0.0,0.0
2,Released,124.0,4.0,100.0,23.0,1.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()